In [ ]:
pip install opencv-contrib-python


In [ ]:
import cv2
import sys

In [1]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow


  Using cached tensorflow-2.11.0-cp39-cp39-win_amd64.whl (1.9 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf

In [1]:
import os
import time
import pandas as pd
import numpy as np

In [3]:
pip install tensorflow_hub

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
import tensorflow_hub as hub

In [4]:
def load_dataset(path, num_samples):
    #read data, use columns 6 and 9 which representing score and text
    df = pd.read_csv(path, usecols=[1,2], nrows=num_samples)
    df.columns = ['rating', 'title']
    
    text = df['title'].tolist() # add each title to list
    #type cast elements in list to string, encode in ascii
    text = [str(t).encode('ascii', 'replace') for t in text]
    #convert to numpy array
    text = np.array(text,dtype=object)[:]
    
    labels = df['rating'].tolist()
    #preprocessing - map 5 star rating ssytem to three classes: negative, positive, and neutral
    # 4-5 maped to 1, 3 mapped ot 0, 2-1 mapped to -1
    labels = np.array(pd.get_dummies(labels),dtype=int) #convert to array, one hot encode
    for i in labels:
        if i in ["enthusiasm", "fun", "happiness", "love", "surprise", "relief"]:
            i = 0
        elif i in ["neutral", "bored"]:
            i = 1
        elif i == "hate":
            i = 2
        elif i == "empty":
            i = 3
        elif i == "sadness":
            i = 4
        elif i == "worry":
            i = 5
        elif i == "anger":
            i = 6
    return labels, text  

tmp_labels, tmp_text = load_dataset('test.csv', 15)
tmp_labels

C:\Users\Rawan Alamily\AppData\Local\Temp\ipykernel_7188\698571244.py:17: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if i in ["enthusiasm", "fun", "happiness", "love", "surprise", "relief"]:
C:\Users\Rawan Alamily\AppData\Local\Temp\ipykernel_7188\698571244.py:19: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif i in ["neutral", "bored"]:
C:\Users\Rawan Alamily\AppData\Local\Temp\ipykernel_7188\698571244.py:21: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif i == "hate":
C:\Users\Rawan Alamily\AppData\Local\Temp\ipykernel_7188\698571244.py:23: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif i == "empty":
C:\Users\Rawan Alamily\AppData\Local\Temp\ipykern

array([[0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0]])

In [5]:
def get_model():
    # pretrained text embedding model  -maps input strings to 50 dimensional vectors, which we feed into 3 layer softmax that classifies into our 3 classes 
    hub_layer = hub.KerasLayer('https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1', 
                              output_shape=[128], input_shape=[], dtype=tf.string, name='input', 
                              trainable=False)  # we are using a pretrained module, thereforre training is disabled
    # Keras model architechture 
    model = tf.keras.Sequential()
    model.add(hub_layer)
    model.add(tf.keras.layers.Dense(64, activation='relu')) # 164 hidden layers, relu activation function
    model.add(tf.keras.layers.Dense(7, activation='softmax', name='output')) # 3 hidden, 
    
    model.compile(loss='sparse_categorical_crossentropy', 
                 optimizer='Adam',
                 metrics=['accuracy'])    
    model.summary()
    return model
    

In [6]:
embed = hub.load('https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1')
embeddings = embed(['this is a test', 'look at the embeddings'])
embeddings

<tf.Tensor: shape=(2, 50), dtype=float32, numpy=
array([[ 0.05650096,  0.2567145 ,  0.24404189,  0.14395264, -0.05569138,
        -0.10513686,  0.09544804,  0.3080969 , -0.218672  , -0.03048538,
        -0.19036277,  0.01005417,  0.11541115, -0.14860378,  0.03914931,
        -0.2561884 , -0.15442336,  0.12836292,  0.0469152 , -0.1500514 ,
        -0.13068351, -0.01958708,  0.09192695,  0.1208052 , -0.12291992,
        -0.04548305, -0.3679261 ,  0.05125156,  0.09797382, -0.10217863,
        -0.1965521 ,  0.15523128, -0.05881735, -0.16426983,  0.06646369,
         0.05789638,  0.15421619, -0.24014738,  0.11075415, -0.10756174,
        -0.01679449, -0.01877424,  0.18602087,  0.2623015 , -0.3829217 ,
        -0.34895867, -0.0868978 ,  0.02295742,  0.03787762, -0.02646483],
       [-0.01533648,  0.2517981 ,  0.15771465,  0.10011643, -0.03027005,
        -0.09655963,  0.10035348, -0.13405894, -0.13515756,  0.15999079,
        -0.0257801 ,  0.01482286,  0.17336626,  0.02416893, -0.02589497,
 

In [7]:
def train(EPOCHS=5, BATCH_SIZE=32, TRAIN_FILE='train.csv', VAL_FILE='test.csv'):
    WORKING_DIR = os.getcwd() #gets home directory - avoid hardcoding
    # Load data
    print('Loading training and validation data ...')
    y_train, x_train = load_dataset(TRAIN_FILE, 100000)
    y_val, x_val = load_dataset(VAL_FILE, 100000)
    y_train = tf.keras.utils.to_categorical(y_train, 7)
    y_val = tf.keras.utils.to_categorical(y_val, 7)
    # Train
    print('Training the model  ...')
    model = get_model()
    model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=1, 
             validation_data =(x_val, y_val),
             callbacks=[tf.keras.callbacks.ModelCheckpoint(os.path.join(WORKING_DIR, 'model_checkpoint'),
                                                          monitor='val_loss', verbose=1,
                                                          save_best_mode=True,
                                                          save_weigths_only=False,
                                                          mode='auto')])
    return model

In [8]:
def export_model(model, base_path='amazon_review/'):
    save_path = os.path.join(base_path, str(int(time.time()))) # append time stamp of model export to the folder we are saving models in
    tf.saved_model.save(model, save_path)

if __name__ =='__main__':
    model = train()
    export_model(model)

Loading training and validation data ...


C:\Users\Rawan Alamily\AppData\Local\Temp\ipykernel_7188\698571244.py:17: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if i in ["enthusiasm", "fun", "happiness", "love", "surprise", "relief"]:
C:\Users\Rawan Alamily\AppData\Local\Temp\ipykernel_7188\698571244.py:19: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif i in ["neutral", "bored"]:
C:\Users\Rawan Alamily\AppData\Local\Temp\ipykernel_7188\698571244.py:21: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif i == "hate":
C:\Users\Rawan Alamily\AppData\Local\Temp\ipykernel_7188\698571244.py:23: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif i == "empty":
C:\Users\Rawan Alamily\AppData\Local\Temp\ipykern

Training the model  ...


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (KerasLayer)          (None, 128)               124642688 
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 output (Dense)              (None, 7)                 455       
                                                                 
Total params: 124,651,399
Trainable params: 8,711
Non-trainable params: 124,642,688
_________________________________________________________________
Epoch 1/5


ValueError: in user code:

    File "C:\Users\Rawan Alamily\anaconda3\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Rawan Alamily\anaconda3\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Rawan Alamily\anaconda3\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Rawan Alamily\anaconda3\lib\site-packages\keras\engine\training.py", line 1028, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "C:\Users\Rawan Alamily\anaconda3\lib\site-packages\keras\engine\training.py", line 1122, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "C:\Users\Rawan Alamily\anaconda3\lib\site-packages\keras\engine\compile_utils.py", li

In [17]:
test_sentance = "Im anxious"
model.predict([test_sentance])

1/1 [==============================] - 0s 75ms/step


array([[0.08568654, 0.08717445, 0.827139  ]], dtype=float32)

In [96]:
# THERAPY BOT RESPONSES 


# the model outputs an array of size 1 to 7, each index representing an emothion as defined in 
# our load_dataset function
# the index with the largest value is chosen to be the main emotion the individual is feeling, 
# and with that information our chatbot responds accordingly with the statments below:

result = [0.0, 0.4, 0.001, 0.7, 0.99] # example result array
emotion_val = 0
emotion_index = 0

for i in range(len(result)):
    if result[i] > emotion_val:
        emotion_val = result[i]
        emotion_index = i

if i == 0:
     print ("Glad you're having fun")
elif i == 1:
    print ("Try to do something you enjoy in the next few days")
elif i == 2:
    print("Step away and remember why you value this and what caused you to feel this way")
elif i == 3:
    print("Try to set goals and strive for something new!")
elif i == 4:
    print("I suggest you reach out to someone you trust")
elif i == 5:
    print("Things will work out, as they always do. Just take a deep breath and take it slow.")
elif i == 6:
    print("Take some time off and do something you enjoy to blow off some steam")

I suggest you reach out to someone you trust
